
# Implementación con _multiprocessing_

El algoritmo inicialmente se implementó con un enfoque secuencial en la clase `colony` para la solución del problema TSP con el algoritmo ACO. Con el fin de volver más eficiente el código y reducir el tiempo de ejecución se implementó multi-procesamiento para hacer _spawn_ y crear un _pool_ de procesos utilizando la librería [`multiprocessing`](https://docs.python.org/3/library/multiprocessing.html) de `Python`.

Se incluyó la clase `colony_mw` en el modulo [aco_tsp_oo](https://github.com/optimizacion-2-2021-1-gh-classroom/practica-1-segunda-parte-ltejadal/blob/main/src/ant_colony/aco_tsp_oo.py) del repo original que contiene la librería. El funcionamiento y lógica de esta función se explican a detalle en el siguiente [`notebook`](https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-ltejadal/blob/main/notebooks/eficiencia_codigo/reimplementacion_multiprocessing.ipynb).

El propósito de este `notebook` es presentar la utilización de la nueva clase utilizando 1000 hormigas para resolver el problema TSP de 100 ciudades chinas, sacadas aleatoriamente del conjunto de la base de datos National Traveling Salesman Problems de la Universidad de Waterloo, disponible [aquí](https://www.math.uwaterloo.ca/tsp/world/countries.html). Adicionalmente, se hace una comparación de tiempo de computo con respecto a la clase `colony`. **En esta pruba se obtiene una mejora importante en el tiempo de cómputo con la nueva implementación**.

Para efectos de comparabilidad incluimos las características de la máquina donde se ejecuta esta prueba.

In [6]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              12
On-line CPU(s) list: 0-11
Thread(s) per core:  2
Core(s) per socket:  6
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               165
Model name:          Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz
Stepping:            2
CPU MHz:             2191.606
CPU max MHz:         5000.0000
CPU min MHz:         800.0000
BogoMIPS:            5199.98
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            12288K
NUMA node0 CPU(s):   0-11
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq dtes64 monitor ds_cpl vmx est tm2 ssse3

In [1]:
# instalar libreria
!pip install "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-1-segunda-parte-ltejadal.git#egg=ant-colony&subdirectory=src" &> /dev/null

In [7]:
import ant_colony as ac
import time
n_ants = 1000
n_cities =100

In [8]:
path_china = '../../datasets/ch71009.tsp'
G = ac.read_coord_data(path_china, n_cities=n_cities, seed=1999)

Problem with 71009 cities. Selected 100.


## Implementación con clase `colony_multiw`

Inicialmente instanciamos la nueva clase, y para este ejemplo se utilizaŕa la totalidad de CPU disponibles en la máquina, 12, y 1000 hormigas.

In [9]:
colony_mw = ac.colony_multiw(G, init_node=0,  n_ants= n_ants, n_workers=12)

In [10]:
start_time = time.time()
colony_mw.solve_tsp()
end_time = time.time()

In [11]:
secs = end_time-start_time
print("La solucion con pool de workers tomó", secs, "segundos." )
print(f"Distancia {colony_mw.best_dist} kms.")

La solucion con pool de workers tomó 21.603434801101685 segundos.
Distancia 1119.0648040721273 kms.


## Comparación con clase `colony`

Para la utilización de la clase `colony` solo basta con indicar el número de hormigas que va a utilizar la colonia, 1000, dado que se realizará computo secuencial con 1 CPU. 

In [12]:
colony_old = ac.colony(G, init_node=0,  n_ants= n_ants)

In [13]:
start_time = time.time()
colony_old.solve_tsp()
end_time = time.time()

In [14]:
secs = end_time-start_time
print("La solucion sin pool tomó", secs, "segundos." )
print(f"Distancia {colony_old.best_dist} kms.")

La solucion sin pool tomó 794.7479469776154 segundos.
Distancia 1119.0648040721273 kms.


## Conclusión

Como se puede observar, con la nueva implementación que incorpora paralelismo en el recorrido de las hormigas **redujo el tiempo de cómputo aproximadamente 37 veces**, pasando de ser 794.74 segundos a 21.6 segundos. 

## Referencias

- Capítulo V del libro de optimización (Cómputo en paralelo usando CPUs en un sistema de memoria compartida (SMC): https://itam-ds.github.io/analisis-numerico-computo-cientifico/V.optimizacion_de_codigo/5.4/Computo_en_paralelo_usando_CPUS_en_SMC.html#
- Librería multiprocessing: https://docs.python.org/3.1/library/multiprocessing.html
